#NLP part 4
## CMPE 257 - Machine Learning - Homework NLP Part 4 - Loading and Querying Pretrained Mistral7B Model, Analysis of the Generated Poems.


**Team Name** : Code Wizards <br/><br/>
**Team Members** : <br/>
Bhavya Hegde <br>
Blessy Dickson Daniel Moses <br/>
Darshini Venkatesha Murthy Nag <br/>
Sirisha Polisetty <br/>


In [ ]:
# # Installing necessary libraries for NLP, data analysis, web development, and other tasks.
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas
!pip install bertopic
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml
!pip install fastapi kaleido python-multipart uvicorn
!pip install numpy==1.23.3 pandas==1.5.3
!pip install datasets

  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached pandas-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.3
    Uninstalling numpy-1.23.3:
      Successfully uninstalled numpy-1.23.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.2 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.3 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
  Using cached 

In [ ]:
# # Importing libraries for NLP, web scraping, data analysis, and Google Colab drive mounting.
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


In [ ]:
# Suppressing specific warning messages to maintain clear and concise output.
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Downloading necessary NLTK data packages for text processing.
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
poet = "William Shakespeare"
store_loc = "/content/drive/MyDrive/LLM-Poem/"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining functions to clean text and scrape poems from URLs.
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        #poems_results.append([poem_title, poem_for_machine, poem_for_human])
        poems_results.append([poem_title, poem_for_machine])
    return poems_results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to retrieve poem URLs for a specific poet.
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 30:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 30:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd_title_list = ["Title", "Poem"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Defining a function to get poems and save them to a CSV file.
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
        temp_name = file_name
        file_name_json = temp_name.replace(".csv",".json")
        df.to_json(file_name_json)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
file_name = store_loc + poet.replace(" ", "_") + ".csv"
#print(file_name)
data= getPoem(file_name, get_poem_url_json(poet)['URL'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Title,Poem
0,0,Sonnet 116: 'Let me not to the marriage of tru...,let me not to the marriage of true minds admit...
1,1,All The World's A Stage,all the world stage and all the men and women ...
2,2,Witches' Chant (from Macbeth),1st witch thrice the brinded cat hath mew 2nd ...
3,3,"Blow, Blow, Thou Winter Wind",blow blow thou winter wind thou art not so unk...
4,4,Sonnet 130: 'My mistress' eyes are nothing lik...,my mistress eyes are nothing like the sun cora...
5,5,Spring And Winter,when icicles hang by the wall and dick the she...
6,6,Carpe Diem,mistress mine where are you roaming stay and h...
7,7,The Dark Lady Sonnets (127 - 154),cxxviiin the old age black was not counted fai...
8,8,Sigh No More,sigh no more ladies sigh no more men were dece...
9,9,Under The Greenwood Tree,under the greenwood tree who loves to lie with...


In [ ]:
# Defining a function to get poems and save them to a CSV file.
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
        temp_name = file_name
        file_name_json = temp_name.replace(".csv",".json")
        df.to_json(file_name_json)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def formatting_func(example, chunk_type='poem'):
    poet_name = "William Shakespeare"
    text = ""

    if chunk_type == 'poem':
        # Formatting for the entire poem
        text = f"### The following is a poem by the poet {poet_name}:\n{example['Poem']}"
    elif chunk_type == 'line':
        # Formatting for each line of the poem
        lines = example['Poem'].split('\n')  # Assuming each line of the poem is separated by a newline character
        text = f"### The following are lines by the poet {poet_name}:\n"
        text += '\n'.join([f"- {line}" for line in lines if line.strip() != ''])
    else:
        raise ValueError("chunk_type must be either 'poem' or 'line'.")

    return text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#**Load the base model**

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops sentencepiece

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Load and Query the Pre-Trained Mistral7BV1 Model

In [ ]:
!pip install transformers

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from huggingface_hub import notebook_login
!huggingface-cli login

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
import torch, transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
trained_model = AutoModelForCausalLM.from_pretrained("darshini/MistralAI7BV1", quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
trained_tokenizer = AutoTokenizer.from_pretrained(
    "darshini/MistralAI7BV1",
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
trained_tokenizer.pad_token = trained_tokenizer.eos_token
trained_model.eval()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (k_proj): Linear4bit(
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_featur

In [ ]:
from transformers import pipeline,logging

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
poems = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Generate a poem that could be written by William Shakespeare who was one of the best poets in the World:"
pipe = pipeline(task="text-generation", model=trained_model, tokenizer=trained_tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
poems.append(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<s>[INST] Generate a poem that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Upon the canvas of the midnight sky,
The stars do dance in a celestial ballet,
Their light, a sonnet sung by the lyrical sigh,
Of love and longing that will never sway.

The moon, a silver coin in the inky sea,
Its glow illuminates the world below,
And in its light, our dreams are set free,
To soar and glide where our hearts dare to go.

But all must fade, as the dawn doth approach,
And the night's enchantment begins to wane,
Yet in our hearts, the memory we'll probe,
Of a love that was born 'neath the starry rain.

So let us cherish the moments we share,
In the glow of the moon's gentle, silver light.


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem that could be written by William Shakespeare who was one of the best poets in the World:"
pipe = pipeline(task="text-generation", model=trained_model, tokenizer=trained_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
poems.append(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<s>[INST] Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Sure, here's a poem that could be written by William Shakespeare:
In fields of green and skies of blue,
Where love doth bloom anew,
The world is but a stage,
Where we can dance and sway with glee.
The sun and moon above,
Shine down on us with love,
And in each other's arms we'll stay,
Forever and a day.
So let us cherish every day,
And make the most of every play,
For life is but a fleeting dream,
That we must hold on tight and gleam.


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Carpe diem that could be written by William Shakespeare who was one of the best poets in the World:"
pipe = pipeline(task="text-generation", model=trained_model, tokenizer=trained_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
poems.append(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<s>[INST] Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Carpe diem that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Sure, I can help you with that! Here's a poem similar to Carpe diem that could be written by William Shakespeare:
"Seize the day, my heart, and let us fly,
Before the sun doth set and night draws nigh.
The world is ours to conquer and explore,
So let us live with passion and desire to store.
For time is fleeting and our youth will fade,
But in our memories, our love will never wane.
So let us seize the day and make the most of time,
For in the end, it


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Dirge that could be written by William Shakespeare who was one of the best poets in the World:"
pipe = pipeline(task="text-generation", model=trained_model, tokenizer=trained_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
poems.append(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<s>[INST] Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Dirge that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Sure, I can help you with that! Here's a poem similar to Shakespeare's Dirge:

Oh mournful day, when all is lost and gone,
And shadows fall like daggers sharp and keen,
When hope is dead and sorrow doth outweigh
The sweetest joy that e'er was seen or known.

Alas, what cruel hand hath torn me from thee,
My dearest love, my heart's sole joy and light?
What cruel hand hath rent me from thy side,
And left me wandering in this dark


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Bridal song that could be written by William Shakespeare who was one of the best poets in the World:"
pipe = pipeline(task="text-generation", model=trained_model, tokenizer=trained_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
poems.append(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<s>[INST] Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem similar to Bridal song that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Sure, I can help you with that! Here's a poem similar to Bridal Song that could be written by William Shakespeare:

Upon this day of joy and mirth,
Where two hearts beat as one in love,
I sing to thee with all my worth,
My heart's devotion doth prove.

Thy beauty shines like morning sun,
And in thy eyes, a world of wonder,
Thy touch, a sweet and tender one,
My heart's desire, my soul's wonder.

As we embark on this journey true


In [ ]:
for i, poem in enumerate(poems):
    print(f"Poem {i + 1}:\n{poem}\n")


Poem 1:
<s>[INST] Generate a poem that could be written by William Shakespeare who was one of the best poets in the World: [/INST] Upon the canvas of the midnight sky,
The stars do dance in a celestial ballet,
Their light, a sonnet sung by the lyrical sigh,
Of love and longing that will never sway.

The moon, a silver coin in the inky sea,
Its glow illuminates the world below,
And in its light, our dreams are set free,
To soar and glide where our hearts dare to go.

But all must fade, as the dawn doth approach,
And the night's enchantment begins to wane,
Yet in our hearts, the memory we'll probe,
Of a love that was born 'neath the starry rain.

So let us cherish the moments we share,
In the glow of the moon's gentle, silver light.

Poem 2:
<s>[INST] Act as a question and answer AI. You will be provided with a question and you should respond based on your fine-tuning data. Provide the most appropriate answer to the question.Generate a poem that could be written by William Shakespeare wh

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#  Saving the model-generated poems in the Generated Poem folder

In [ ]:
import os
import re

# Define the directory where you want to save the poems
directory = "/content/drive/MyDrive/Generated_Poems/"

# Ensure that the directory exists, if not create it
if not os.path.exists(directory):
    os.makedirs(directory)


pattern1 = r"<s>\[INST\].*?\[/INST\]"
pattern2 = r"Sure,.*?:\s*"

for i, poem in enumerate(poems):

    cleaned_poem = re.sub(pattern1, '', poem, flags=re.DOTALL)

    cleaned_poem = re.sub(pattern2, '', cleaned_poem, flags=re.DOTALL).strip()


    filename = os.path.join(directory, f"Poem_{i + 1}.txt")

    with open(filename, 'w') as file:
        file.write(cleaned_poem)

    print(f"Poem {i + 1}:\n{cleaned_poem}\n")




Poem 1:
Upon the canvas of the midnight sky,
The stars do dance in a celestial ballet,
Their light, a sonnet sung by the lyrical sigh,
Of love and longing that will never sway.

The moon, a silver coin in the inky sea,
Its glow illuminates the world below,
And in its light, our dreams are set free,
To soar and glide where our hearts dare to go.

But all must fade, as the dawn doth approach,
And the night's enchantment begins to wane,
Yet in our hearts, the memory we'll probe,
Of a love that was born 'neath the starry rain.

So let us cherish the moments we share,
In the glow of the moon's gentle, silver light.

Poem 2:
In fields of green and skies of blue,
Where love doth bloom anew,
The world is but a stage,
Where we can dance and sway with glee.
The sun and moon above,
Shine down on us with love,
And in each other's arms we'll stay,
Forever and a day.
So let us cherish every day,
And make the most of every play,
For life is but a fleeting dream,
That we must hold on tight and gleam.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Title,Poem
0,0,Sonnet 116: 'Let me not to the marriage of tru...,let me not to the marriage of true minds admit...
1,1,All The World's A Stage,all the world stage and all the men and women ...
2,2,Witches' Chant (from Macbeth),1st witch thrice the brinded cat hath mew 2nd ...
3,3,"Blow, Blow, Thou Winter Wind",blow blow thou winter wind thou art not so unk...
4,4,Sonnet 130: 'My mistress' eyes are nothing lik...,my mistress eyes are nothing like the sun cora...
5,5,Spring And Winter,when icicles hang by the wall and dick the she...
6,6,Carpe Diem,mistress mine where are you roaming stay and h...
7,7,The Dark Lady Sonnets (127 - 154),cxxviiin the old age black was not counted fai...
8,8,Sigh No More,sigh no more ladies sigh no more men were dece...
9,9,Under The Greenwood Tree,under the greenwood tree who loves to lie with...


#Cosine Similarity between model generated and scraped poem written by William Shakespeare

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

csv_file_path = "/content/drive/MyDrive/LLM-Poem/William_Shakespeare.csv"
text_file_path = "/content/drive/MyDrive/Generated_Poems/Poem_1.txt"
poem_column = "Poem"
title_column = "Title"

# Function to read a poem from a text file
def read_poem_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Read the poem from the text file
poem_from_txt = read_poem_from_txt(text_file_path)

# Read all poems from the CSV file
df = pd.read_csv(csv_file_path)

# Initialize a CountVectorizer
vectorizer = CountVectorizer()

# Store similarity scores, poem indices, and titles
similarity_scores_with_titles = []

# Compare with each poem in the CSV
for index, row in df.iterrows():
    poem_from_csv = row[poem_column]
    title = row[title_column]  # Fetch the title
    poem_vectors = vectorizer.fit_transform([poem_from_csv, poem_from_txt])
    cosine_sim = cosine_similarity(poem_vectors)
    similarity_score = cosine_sim[0, 1]
    similarity_scores_with_titles.append((similarity_score, title))

# Sort the scores and select the top 5
top_5_similar_poems = sorted(similarity_scores_with_titles, reverse=True)[:5]


for score, title in top_5_similar_poems:
    print(f"Poem Title: {title},Similarity Score: {score}")


Poem Title: To be, or not to be: that is the question,Similarity Score: 0.6880653667878013
Poem Title: Spring And Winter,Similarity Score: 0.6374575764473416
Poem Title: A Lover's Complaint,Similarity Score: 0.6206109115584875
Poem Title: Sonnet 73: "That time of year thou mayst in me behold...",Similarity Score: 0.5929476500990951
Poem Title: The Quality Of Mercy,Similarity Score: 0.5552857336446748


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Conclusion

The comparison between a poem generated by model and poems by William Shakespeare shows that they are quite similar. The highest similarity score is around 0.68, which means the model generated  poem sounds a lot like Shakespeare's writing. This shows that the model did a good job of writing in a way that's close to how Shakespeare wrote, using similar words and ideas.